Test Notebook for Gary on Excel Creation 

Version 1.0
Author: Gary Manley
Date: 2025-12-23

Setup the imports 

In [1]:
import sys
import os
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add the 'src' directory to the path so we can import the wrapper class
# Assumes this notebook is in the 'notebooks/' folder and source is in 'src/'
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from enterprise_writer import EnterpriseExcelWriter
from query_library import fGetRegionalSales, fGetDataDictionary, fGetRunbyMonth
from config_provider import fGetReportConfig

In [2]:
# 1. Fetch Configuration
vProfileName = 'GaryTest'

# Fetch config from DB
vConfig = fGetReportConfig(vProfileName)


In [3]:
dfRun = fGetRunbyMonth()
vTotalRun = dfRun['total_distance'].sum()

In [4]:
dfDict = fGetDataDictionary()

In [5]:
vReport = EnterpriseExcelWriter("GaryTest.xlsx", vConfig=vConfig)
vReport.fSetColumnMapping(dfDict)

vReport.fAddTitle("Gary Performance Report")
# Example: Adding a note with specific bold/coloured words
vReport.fAddText(["Gary ", {'text': 'Performance ', 'bold': True, 'colour': 'red'},  "Report"],vFontSize=12, vFontColour = 'FF0000',vBgColour='FFFF11')
# Add a sensitivity banner at the top
vReport.fAddBanner("OFFICIAL - SENSITIVE - CONFIDENTIAL", vStyleProfile='Warning')
vReport.fAddKpiRow({'Total Distance Run': vTotalRun})
vReport.fSkipRows(1)

vReport.fAddSeabornChart(dfRun, vXCol='run_month', vYCol='total_distance', vTitle='Run Volume', vRow=7)
vReport.fAddSeabornChart(
    dfRun, 
    vXCol='run_month', 
    vYCol='total_distance', 
    vTitle='Run Volume', 
    vRow=47, 
    vChartType='line'
)
vReport.fSkipRows(20)
vReport.fWriteDataframe(dfRun, vAddTotals=True, vAutoFilter=True)
vReport.fAddConditionalFormat('total_distance', 'cell', {'criteria': '<', 'value': 500000})

    
# Tab 2: Appendix
vReport.fNewSheet("Appendix", "Detailed definitions")
    
# This uses the specific 'DataDict' config component for styling headers
vReport.fAddDataDictionary(dfDict)
dfFilteredDict = vReport.fFilterDataDictionary(dfDict)
vReport.fWriteRichDataframe(dfFilteredDict)

# Works as expected but need to ensure only the right 2 columns are present. 
vReport.fAddDefinitionList(dfFilteredDict)
vReport.fSetColumnWidths({4: 50}) 
vReport.fGenerateTOC()

vReport.fClose() 

File saved: GaryTest.xlsx
